In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import pickle

In [ ]:
data = pd.read_json('/content/drive/MyDrive/Copy of published_jobs_for_job_category_extraction.json')

ValueError: ignored

In [ ]:
data.dropna(axis=0, inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49893 entries, 0 to 49999
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            49893 non-null  object
 1   title         49893 non-null  object
 2   job_category  49893 non-null  object
 3   raw_content   49893 non-null  object
dtypes: object(4)
memory usage: 1.9+ MB


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
JOB_CATEGORY_MIN = 250

In [ ]:
import pandas as pd

pd.set_option('display.max_rows', None)

dataset = pd.read_json('/content/drive/MyDrive/Copy of published_jobs_for_job_category_extraction.json')

dataset = dataset.iloc[:, [1,2]]

dataset.dropna(axis=0, inplace=True)

dataset_copy = dataset.copy()

counts_job_category = dataset_copy['job_category'].value_counts()

# dataset_after_minimum_category 

data = dataset_copy.loc[dataset_copy['job_category'].isin(counts_job_category.index[counts_job_category > JOB_CATEGORY_MIN])]

print(data.shape)

(42241, 2)


In [ ]:
sample_size = 280

In [ ]:
data = data.groupby('job_category').apply(lambda x: x.sample(sample_size) if len(x) > 280 else x.sample(len(x)))

In [ ]:
x = data['title']
y = data['job_category']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.10, random_state=0,stratify=y)

In [ ]:
pd.set_option('display.max_rows', None)
y.value_counts().to_frame()
len(y.unique())

50

In [ ]:
import re
import pandas as pd
import spacy
from bs4 import BeautifulSoup
from spacy.lang.en import STOP_WORDS

# stopwords = stopwords()


nlp = spacy.load('en_core_web_sm')


class EnglishPreprocessor:
    
    def replace_character(self, x_data):
      x_data = x_data.replace({'/': ' ', '&': ' ', ',': ' ', '$': ' ', '\n': ' ', '-':' '}, regex=True)
      return x_data

    def remove_space(self, x_data):
        return x_data.apply(lambda x: " ".join(x.split()))

    def remove_punctuations(self, x_data):
        return x_data.apply(lambda x: re.sub('[^A-Z a-z 0-9]+', '', x))

    def to_lower(self, x_data):
        return x_data.apply(lambda x: x.lower())

    def preprocess(self, x_data):
        # Replace Character
        x_data = self.replace_character(x_data)
        # Remove Punctuation and Special Character
        x_data = self.remove_punctuations(x_data)
        # Remove Multiple Space
        x_data = self.remove_space(x_data)
        # To_lower
        x_data = self.to_lower(x_data)

        return x_data

In [ ]:
x= EnglishPreprocessor().preprocess(x)

In [ ]:
encoder = LabelEncoder()
encoder.fit(y)
encoded_y = encoder.transform(y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_y)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, dummy_y, test_size=0.10, random_state=0,stratify=y)

In [ ]:
from keras.preprocessing import text, sequence

In [ ]:
tokenize = text.Tokenizer()

In [ ]:
tokenize.fit_on_texts(list(x))

In [ ]:
word_index=tokenize.word_index

In [ ]:
vocab_size=len(word_index)

In [ ]:
sentence_train1=tokenize.texts_to_sequences(x_train)

In [ ]:
Input_train=sequence.pad_sequences(sentence_train1,5)

In [ ]:
sentence_train2 = tokenize.texts_to_sequences(x_test)

In [ ]:
Input_test=sequence.pad_sequences(sentence_train2,5)

In [ ]:
def Pretrained_embedding(emb_file):
  embedding_index={}
  with open(emb_file,errors ='ignore',encoding='utf-8') as f:
    for line in f:
      try:

        values=line.split()
        word=values[0]
        vector=np.asarray(values[1:],dtype="float32")
        embedding_index[word]=vector
      except Exception as e:
        continue
  return embedding_index

In [ ]:
embedding_index= Pretrained_embedding("/content/drive/MyDrive/Copy of glove.840B.300d.txt")

In [ ]:
def embedding(word_index,embedding_ind):
    embedding_matrix=np.zeros((len(word_index)+1,300))
    for word,i in word_index.items():
        embedding_vector=embedding_ind.get(word)
        if embedding_vector is not None:
            embedding_matrix[i]=embedding_vector
    return embedding_matrix

In [ ]:
embedding_matrix = embedding(word_index, embedding_index)

In [ ]:
# def create_model(unit,activation,optimizer):
#   ann = Sequential()

#   ann.add(tf.keras.layers.Embedding(
#     input_dim= (len(word_index))+1, output_dim= 300,weights=[embedding_matrix],
#      input_length=5,trainable=False,))

#   ann.add(Bidirectional(LSTM(units=unit, activation=activation,dropout=0.2,recurrent_dropout=0.2,go_backwards=True)))
# # ann.add(Bidirectional(LSTM(50, activation='tanh',go_backwards=True)))
# # ann.add(Dense(units=5, activation='relu'))
# # ann.add(Dropout(0.2))
#   ann.add(Dense(units=unit, activation=activation))
#   ann.add(Dropout(0.2))
# # ann.add(Dense(units=20, activation='softmax'))
#   ann.add(Dense(50, activation='softmax'))
#   ann.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics="accuracy")
#   return ann

In [ ]:
# from keras.layers import *

In [ ]:
# from keras.wrappers.scikit_learn import KerasClassifier
# from sklearn.model_selection import GridSearchCV

In [ ]:
# model = KerasClassifier(build_fn=create_model)

In [ ]:
# optimizers = ['rmsprop', 'adam']
# activation=["relu","tanh","sigmoid"]

In [ ]:
# epochs = np.array([20])
# batches = np.array([5, 10, 20])
# unit=np.array([10,20,30,40,50])

In [ ]:
# param_grid = dict(optimizer=optimizers, nb_epoch=epochs, batch_size=batches,unit=unit,activation=activation )

In [ ]:
# param_grid = dict(optimizer=optimizers, nb_epoch=epochs, batch_size=batches, init=init)
# grid = GridSearchCV(estimator=model, param_grid=param_grid)
# grid_result = grid.fit(X_train, Y_train)

In [ ]:
# grid_result = grid.fit(Input_train,y_train,validation_data=(Input_test,y_test))

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Embedding, Dense, Conv1D, GlobalMaxPooling1D, Concatenate, Dropout


class TextCNN(Model):

    def __init__(self,
                 maxlen,
                 max_features,
                 embedding_dims,
                 kernel_sizes=[3,4,5],
                 class_num=50,
                 last_activation='softmax'):
        super(TextCNN, self).__init__()
        self.maxlen = maxlen
        self.max_features = max_features
        self.embedding_dims = embedding_dims
        self.kernel_sizes = kernel_sizes
        self.class_num = class_num
        self.last_activation = last_activation
        self.embedding = Embedding(self.max_features, self.embedding_dims, input_length=self.maxlen)
        self.convs = []
        self.max_poolings = []
        for kernel_size in self.kernel_sizes:
            self.convs.append(Conv1D(8, kernel_size, activation='relu'))
            self.max_poolings.append(GlobalMaxPooling1D())
        self.classifier = Dense(self.class_num, activation=self.last_activation)

    def call(self, inputs):
        if len(inputs.get_shape()) != 2:
            raise ValueError('The rank of inputs of TextCNN must be 2, but now is %d' % len(inputs.get_shape()))
        if inputs.get_shape()[1] != self.maxlen:
            raise ValueError('The maxlen of inputs of TextCNN must be %d, but now is %d' % (self.maxlen, inputs.get_shape()[1]))
        # Embedding part can try multichannel as same as origin paper
        embedding = self.embedding(inputs)
        convs = []
        for i in range(len(self.kernel_sizes)):
            c = self.convs[i](embedding)
            c = self.max_poolings[i](c)
            convs.append(c)
        x = Concatenate()(convs)
        output = self.classifier(x)
        return output

In [ ]:
model = TextCNN(5, len(word_index)+1, 300)
model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])

In [ ]:
vocab_size=len(word_index)+1

In [ ]:
maxlen = 5

In [ ]:
# embed_dim = 300  # Embedding size for each token
# num_heads = 2  # Number of attention heads
# ff_dim = 32  # Hidden layer size in feed forward network inside transformer

# inputs = layers.Input(shape=(maxlen,))
# embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
# x = embedding_layer(inputs)
# transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
# x = transformer_block(x)
# x = layers.GlobalAveragePooling1D()(x)
# x = layers.Dropout(0.1)(x)
# x = layers.Dense(20, activation="relu")(x)
# x = layers.Dropout(0.1)(x)
# outputs = layers.Dense(50, activation="softmax")(x)

# model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:

# history = model.fit(Input_train,y_train, batch_size=20, epochs=50, validation_data=(Input_test,y_test))

In [ ]:
# CNN=loss: 1.1180 - accuracy: 0.7216 - val_loss: 1.4513 - val_accuracy: 0.6507

In [ ]:
from tensorflow.keras import backend as K
from tensorflow.keras import initializers, regularizers, constraints
from tensorflow.keras.layers import Layer


class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        """
        Keras Layer that implements an Attention mechanism for temporal data.
        Supports Masking.
        Follows the work of Raffel et al. [https://arxiv.org/abs/1512.08756]
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        Example:
            # 1
            model.add(LSTM(64, return_sequences=True))
            model.add(Attention())
            # next add a Dense layer (for classification/regression) or whatever...
            # 2
            hidden = LSTM(64, return_sequences=True)(words)
            sentence = Attention()(hidden)
            # next add a Dense layer (for classification/regression) or whatever...
        """
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0

        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight(name='{}_W'.format(self.name),
                                 shape=(input_shape[-1],),
                                 initializer=self.init,
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight(name='{}_b'.format(self.name),
                                     shape=(input_shape[1],),
                                     initializer='zero',
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        e = K.reshape(K.dot(K.reshape(x, (-1, features_dim)), K.reshape(self.W, (features_dim, 1))), (-1, step_dim))  # e = K.dot(x, self.W)
        if self.bias:
            e += self.b
        e = K.tanh(e)

        a = K.exp(e)
        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())
        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        a = K.expand_dims(a)

        c = K.sum(a * x, axis=1)
        return c

    def compute_output_shape(self, input_shape):
        return input_shape[0], self.features_dim

In [ ]:

from tensorflow.keras import Model
from tensorflow.keras.layers import Embedding, Dense, Bidirectional
from tensorflow.compat.v1.keras.layers import CuDNNLSTM


class TextAttBiRNN(Model):
    def __init__(self,
                 maxlen,
                 max_features,
                 embedding_dims,
                 class_num=50,
                 last_activation='sigmoid'):
        super(TextAttBiRNN, self).__init__()
        self.maxlen = maxlen
        self.max_features = max_features
        self.embedding_dims = embedding_dims
        self.class_num = class_num
        self.last_activation = last_activation
        self.embedding = Embedding(self.max_features, self.embedding_dims, input_length=self.maxlen)
        self.bi_rnn = Bidirectional(LSTM(128, return_sequences=True))  # LSTM or GRU
        self.attention = Attention(self.maxlen)
        self.classifier = Dense(self.class_num, activation=self.last_activation)

    def call(self, inputs):
        if len(inputs.get_shape()) != 2:
            raise ValueError('The rank of inputs of TextAttBiRNN must be 2, but now is %d' % len(inputs.get_shape()))
        if inputs.get_shape()[1] != self.maxlen:
            raise ValueError('The maxlen of inputs of TextAttBiRNN must be %d, but now is %d' % (self.maxlen, inputs.get_shape()[1]))
        embedding = self.embedding(inputs)
        x = self.bi_rnn(embedding)
        x = self.attention(x)
        output = self.classifier(x)
        return output


In [ ]:
model = TextAttBiRNN(5, len(word_index)+1, 300)

In [ ]:
model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history2 = model.fit(Input_train,y_train, batch_size=20, epochs=50, validation_data=(Input_test,y_test))

Epoch 1/50
630/630 [==============================] - 23s 32ms/step - loss: 2.8666 - accuracy: 0.3169 - val_loss: 1.4239 - val_accuracy: 0.6414
Epoch 2/50
630/630 [==============================] - 19s 30ms/step - loss: 1.2890 - accuracy: 0.6662 - val_loss: 1.3044 - val_accuracy: 0.6564
Epoch 3/50
630/630 [==============================] - 19s 30ms/step - loss: 0.9981 - accuracy: 0.7303 - val_loss: 1.3212 - val_accuracy: 0.6536
Epoch 4/50
630/630 [==============================] - 19s 30ms/step - loss: 0.8991 - accuracy: 0.7532 - val_loss: 1.3596 - val_accuracy: 0.6471
Epoch 5/50
630/630 [==============================] - 19s 30ms/step - loss: 0.8255 - accuracy: 0.7727 - val_loss: 1.4080 - val_accuracy: 0.6343
Epoch 6/50
630/630 [==============================] - 19s 30ms/step - loss: 0.7378 - accuracy: 0.7910 - val_loss: 1.4017 - val_accuracy: 0.6486
Epoch 7/50
630/630 [==============================] - 19s 30ms/step - loss: 0.7076 - accuracy: 0.7951 - val_loss: 1.4491 - val_accuracy: